In [1]:
# Package
import logging
logging.getLogger('trimesh').disabled = True
logging.getLogger('shapely.geos').disabled = True
logging.getLogger('matplotlib').setLevel(logging.CRITICAL)
import numpy as np
import matplotlib.pyplot as plt
from pvtrace import *
from pvtrace.data import lumogen_f_red_305
import functools
import trimesh
from trimesh import transformations as trf
import math
import random
import time
from pylab import imread
import pyvista as pv
from pyvista import examples

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
###############################################
# Import Objects
myobj = trimesh.load_mesh("monkey_Suzanne.stl", enable_post_processing=True, solid=True) # https://www.thingiverse.com/thing:1287391
bot   = trimesh.load_mesh("myobj.stl", enable_post_processing=True, solid=True) # Created with creation_exotic_obj_v01.py

In [3]:
###############################################
# Parameters of the geometry
w = 3.0 # width
h = 2.0 # heigh
L = 4.0 # length

###############################################
np.random.seed(1988)    # Keep the same seed for random numbers, at the end, the data will be same for every new simulations
numb_photons = 25       # Number of photons

###############################################
# Dye Lumogen F Red
x = np.linspace(200, 800, 200)  # wavelength, units: nm
absorption_spectrum = lumogen_f_red_305.absorption(x)  # units: nm-1
emission_spectrum = lumogen_f_red_305.emission(x)      # units: nm-1

In [4]:
###############################################
# Transformation
# Get real dimensions of the myobj
w_real  = abs(myobj.bounds[0,0]) + abs(myobj.bounds[1,0])
L_real  = abs(myobj.bounds[0,0]) + abs(myobj.bounds[1,0])
h_real  = abs(myobj.bounds[0,2]) + abs(myobj.bounds[1,2])

# Calculate the scale ratio
f_scal_x = w / w_real
f_scal_y = L / L_real
f_scal_z = h / h_real

# Create matrix for transformation
scale   = np.array([f_scal_x, f_scal_y, f_scal_z])                 # scale : vector of 3 scaling factors
shear   = np.array([0.0, 0.0, 0.0])                 # shear : list of shear factors for x-y, x-z, y-z axes
angles  = np.array([0.0, 0.0, 0.0]) * 0.5*math.pi   # angles : list of Euler angles about static x, y, z axes
trans   = np.array([0.0, 0.0, 0.0])                 # translate : translation vector along x, y, z axes
persp   = np.array([1.0, 1.0, 1.0, 1.0])            # perspective : perspective partition of matrix
       
M0 = trf.compose_matrix(scale, shear, angles, trans, persp)

# Check the transformation before/after
#print(myobj.bounds)
myobj.apply_transform(M0) # Apply transf
bot.apply_transform(M0)
#print(myobj.bounds)

<trimesh.Trimesh(vertices.shape=(7700, 3), faces.shape=(15396, 3))>

In [5]:
###############################################
# Watertight
if myobj.is_watertight == False:
    print('The structure is not watertight !!!')
    trimesh.repair.broken_faces(myobj, color=[255,0,0,255])
    myobj.show(smooth=False)
    # myobj = trimesh.convex.convex_hull(myobj, qhull_options='QbB Pp Qt') #qhull_options='QbB Pp Qt' #http://www.qhull.org/html/qh-quick.htm#options
    #print(trimesh.repair.broken_faces(myobj)) #Index of faces which breake the watertight status of the mesh
    trimesh.repair.fill_holes(myobj)
    trimesh.repair.fix_inversion(myobj)
    trimesh.repair.fix_normals(myobj)
    trimesh.repair.fix_winding(myobj)
    myobj.show(smooth=False)
    if myobj.is_watertight == False:
        print('Error !')

The structure is not watertight !!!
Error !


In [6]:
###############################################
# Environment
world = Node(
    name="world (air)",
    geometry=Sphere(
        radius=20.0,
        material=Material(refractive_index=1.0)
    )
)

#cube = Node(
    #name='cubix',
    #geometry=
        #Box((w, L, h),
        #material=Material(refractive_index=4.0)
        #),
    #parent=world
    #)
        
# Define our geometry
surf = Node(
    name="myobj",
    geometry=Mesh(  
        myobj,
        material=Material(refractive_index=4.0,
                          components=[
                              Luminophore(
                                  coefficient=np.column_stack((x, absorption_spectrum)),
                                  emission=np.column_stack((x, emission_spectrum)),
                                  quantum_yield=1.0,
                                  phase_function=isotropic),
                              Scatterer(coefficient=0.0)]),
                          ),                         
    parent=world
    )

# Define our geometry
pedestal = Node(
    name="pedestal",
    geometry=Mesh(  
        bot,
        material=Material(refractive_index=4.0,
                          components=[
                              Luminophore(
                                  coefficient=np.column_stack((x, absorption_spectrum)),
                                  emission=np.column_stack((x, emission_spectrum)),
                                  quantum_yield=1.0,
                                  phase_function=isotropic),
                              Scatterer(coefficient=0.0)]),
                          ),                         
    parent=world
    )
                              
surf.translate((0, 0, h/2+abs(myobj.bounds[0,2])))
#pedestal.translate((0, 0, h/2+abs(bot.bounds[0,2])))

Node(myobj)

In [7]:
###############################################
# The source of rays
light = Node(
    name="Light ex",
    light=Light(position=functools.partial(rectangular_mask, 0.1, 0.1)),
    parent=world
    )

light.rotate(np.pi*1.0, (0, 1, 0))
light.translate((0, 0, 10))

Node(Light ex)

In [8]:
###############################################
# Renderer: Use meshcat to render the scene (optional)
viewer = MeshcatRenderer(open_browser=True, transparency=True, opacity=0.25, wireframe=True)
scene = Scene(world)
viewer.render(scene)

###############################################
# Iterations of rays
for ray in scene.emit(numb_photons):
    history = photon_tracer.follow(scene, ray, maxsteps=1000)
    path, events = zip(*history)
    viewer.add_ray_path(path) 
    time.sleep(0.5)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
